In [382]:
import numpy as np
import cvxpy as cp
import networkx as nx

from numpy.random import default_rng
from opt_utils import decompose_psd, hyperplane_rounding, complex_hyperplane_rounding, fixed_point_iteration, normalize_rows, load_graph

# Max-cut

In [ ]:
graph_file = "G48.mtx"
graph_type = 1

# graph_file = "toruspm3-8-50.dat"
# graph_type = 0

n = 100
G = load_graph(graph_file, graph_type, n)

In [ ]:
L_val = nx.laplacian_matrix(G).toarray() * 1.0

In [ ]:
sparsity_graph = G.copy()
# TODO: graph needs to be chordal for the PSD theorem to work

In [ ]:
treewidth, tree_decomp = nx.algorithms.approximation.treewidth_min_degree(sparsity_graph)

In [ ]:
nx.draw(tree_decomp, nx.spring_layout(tree_decomp))

In [ ]:
X = []
n_tree = tree_decomp.number_of_nodes()
masks = np.full((n_tree, n, n), False)  # first apply masks to A_i to extract the non-zero terms to be in A_tilde_ik
permutations = []  # then extract the indices correponding to bag k to get A_tilde_ik

for i, bag in enumerate(tree_decomp.nodes):
    X_i_size = len(bag)
    X.append(cp.Variable((X_i_size, X_i_size), PSD=True))
    permutations.append(np.zeros((X_i_size, n)))
    for node_bag_idx, node in enumerate(bag):
        permutations[i][node_bag_idx][node] = 1

In [ ]:
remaining_edges_set = set(G.edges)
remaining_nodes_set = set(G.nodes)
edge_list = list(G.edges)

for k, bag in enumerate(tree_decomp.nodes):
    for i in bag:
        for j in bag:
            if (i, j) in remaining_edges_set:
                masks[k][i][j] = True
                masks[k][j][i] = True
                remaining_edges_set.remove((i, j))

                if i in remaining_nodes_set:
                    masks[k][i][i] = True
                    remaining_nodes_set.remove(i)
                if j in remaining_nodes_set:
                    masks[k][j][j] = True
                    remaining_nodes_set.remove(j)

In [ ]:
# verify the masks add up to the adjacency matrix
np.all(np.sum(masks, axis=0) == nx.to_numpy_array(G, weight=None) + np.eye(n))

In [ ]:
# create constraint matrices
A = [ L_val ]
for i in range(n):
    A_i = np.zeros((n,n))
    A_i[i][i] = 1
    A.append(A_i)

In [ ]:
# constraint matrices for the blocks
A_tilde = []
for i, A_i in enumerate(A):
    A_tilde.append([])
    for k in range(tree_decomp.number_of_nodes()):
        P_k = permutations[k]
        A_tilde[i].append(P_k @ (A_i * masks[k]) @ P_k.T )

In [ ]:
constraints = [ ]

for i in range(1, n + 1):
    constraints += [ cp.sum([ cp.trace(A_tilde[i][k] @ X[k]) for k in range(n_tree) ]) == 1 ]

# consistency constraints
for (j,k) in tree_decomp.edges:
    inter = j.intersection(k)
    inter_permutation = np.zeros((len(inter), n))
    for node_inter_idx, node in enumerate(inter):
        inter_permutation[node_inter_idx][node] = 1
    bag_j_idx = list(tree_decomp.nodes).index(j)
    bag_k_idx = list(tree_decomp.nodes).index(k)
    P_j = permutations[bag_j_idx]
    P_k = permutations[bag_k_idx]
    constraints += [ inter_permutation @ (P_k.T @ X[bag_k_idx] @ P_k - P_j.T @ X[bag_j_idx] @ P_j) @ inter_permutation.T == 0  ]

In [ ]:
prob = cp.Problem(cp.Maximize(1/4 * ( cp.sum([ cp.trace(A_tilde[0][k] @ X[k]) for k in range(n_tree) ]) )), constraints)
prob.solve(solver=cp.MOSEK, verbose=True)

In [ ]:
for i in range(n_tree):
    print(np.linalg.eigvalsh(X[i].value))